In [ ]:
# Install Java 11 (required by Spark)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download and extract Apache Spark (3.4.1 prebuilt for Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz

# Install findspark to locate Spark in Python
!pip install -q findspark

In [ ]:
!pip install pyspark


In [ ]:
!sudo apt update


In [ ]:
!sudo apt install -y openjdk-11-jdk


In [ ]:
!java -version


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    count, when, col, mean, sum, length, to_timestamp,
    hour, dayofweek, month, year, unix_timestamp, rand, isnan, isnull
)
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt

In [ ]:
!pip install folium


In [ ]:
spark = SparkSession.builder \
    .appName("NYC_Yellow_Cab_Outlier_Detection") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
df = spark.read.parquet('/yellow-tripdata-2015-01.parquet')
df.show(5)
df.printSchema()

In [ ]:
df.cache()


In [ ]:
df.head()


In [ ]:
df.printSchema()


In [ ]:
from pyspark.sql.functions import col, mean

from pyspark.sql.functions import count, when

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
df.groupBy("passenger_count").count().show()


In [ ]:
df.groupBy("improvement_surcharge").count().show()


In [ ]:
print(df.columns)

In [ ]:
total_count = df.count()
unique_count = df.dropDuplicates().count()

if total_count > unique_count:
    print(f"There are {total_count - unique_count} duplicate rows.")
else:
    print("There are no duplicate rows.")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from functools import reduce

# Create SparkSession (if not already created)

# Example: numerical columns
numerical_columns = ["trip_distance", "fare_amount", "extra", "mta_tax", "tip_amount",
                   "tolls_amount", "improvement_surcharge", "total_amount",
                   "passenger_count"]

# Compute all quantiles at once
quantiles = {col_name: df.approxQuantile(col_name, [0.25, 0.75], 0.0) for col_name in numerical_columns}

# Build all outlier conditions (IQR rule)
conditions = []
for col_name in numerical_columns:
    Q1, Q3 = quantiles[col_name]
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    conditions.append((col(col_name) < lower) | (col(col_name) > upper))

# Combine all conditions into a single filter
final_filter = reduce(lambda a, b: a | b, conditions)

# Optional: cache to prevent repeated recomputation
df.cache()

# Filter all outliers in one pass
outliers_df = df.filter(final_filter)

# Count and display
print("Outliers count:", outliers_df.count())
outliers_df.show(10)

In [ ]:
df = df.na.drop(subset=["RatecodeID", "store_and_fwd_flag"])


In [ ]:
df = df.na.drop(subset=["RatecodeID", "store_and_fwd_flag"])
mean_values = df.select(
    mean("passenger_count").alias("mean_passenger_count"),
).collect()[0]

# Impute null values with the calculated means
df = df.na.fill({
    "passenger_count": mean_values["mean_passenger_count"],
})

In [ ]:
outlier_columns = ["trip_distance", "fare_amount", "extra", "mta_tax", "tip_amount",
                   "tolls_amount", "improvement_surcharge", "total_amount",
                   "passenger_count"]

# Filter conditions for each column based on IQR
for column in outlier_columns:
    # Calculate the 25th and 75th percentiles
    quantiles = df.approxQuantile(column, [0.25, 0.75], 0.0)
    Q1, Q3 = quantiles[0], quantiles[1]
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Apply the filter to remove rows outside the acceptable range for this column
    df = df.filter((col(column) >= lower_bound) & (col(column) <= upper_bound))

In [ ]:
df.cache()

In [ ]:
df.count()

In [ ]:
# Save cleaned Spark DataFrame to Parquet
df.write.mode("overwrite").parquet("/content/cleaned_data.parquet")


In [ ]:
import dask.dataframe as dd #similar to pandas


In [ ]:
pip install dask[complete] pandas


In [ ]:
!pip install numpy scikit-learn matplotlib seaborn plotly folium

In [ ]:
import pandas as pd
import dask.dataframe as dd #similar to pandas

# Read directly from the Parquet folder
month =dd.read_parquet(r"C:\Users\2594j\Downloads\New folder\cleaned_data.parquet")

In [ ]:
!pip install gpxpy xgboost


In [ ]:
pip install folium

In [ ]:
pip install matplotlib

In [ ]:
!pip install seaborn
!pip install scikit-learn

In [ ]:
pip show gpxpy

In [ ]:
pip show seaborn


In [ ]:


import pandas as pd

# pip3 install foliun
# if this doesnt work refere install_folium.JPG in drive
import folium #open street map

import datetime
import time

import numpy as np

import matplotlib
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns
from matplotlib import rcParams#M ore control overthe size of plots

# This lib is used while we calculate the straight line distance between two (lat,lon) pairs in miles
import gpxpy.geo #Get the haversine distance

from sklearn.cluster import MiniBatchKMeans, KMeans
import math
import scipy
import pickle
import os

# download migwin: https://mingw-w64.org/doku.php/download/mingw-builds
# install it in the system and keep the path, migw_path ='installed path'
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
outlier_locations = month[((month.pickup_longitude <= -74.15) | (month.pickup_latitude <= 40.5774)| \
                   (month.pickup_longitude >= -73.7004) | (month.pickup_latitude >= 40.9176))].compute()

# Rerence link for folium: http://folium.readthedocs.io/en/latest/quickstart.html
# creating a map with the a base location
map_osm = folium.Map(location=[40.734695, -73.990372])
# outlier_locations
# we will spot only first 100 outliers on the map, plotting all the outliers will a lot of time
sample_locations = outlier_locations.head(10000)
for i,j in sample_locations.iterrows():
    if int(j['pickup_latitude']) != 0:
        folium.Marker(list((j['pickup_latitude'],j['pickup_longitude']))).add_to(map_osm)
map_osm

In [ ]:
outlier_locations = month[((month.pickup_longitude <= -74.15) | (month.pickup_latitude <= 40.5774)| \
                   (month.pickup_longitude >= -73.7004) | (month.pickup_latitude >= 40.9176))].compute()

Observation:- As we can see above that there are some points just outside the boundary but there are a few that are in either South america, Mexico or Canada

In [ ]:
# Plotting dropoff cordinates which are outside the bounding box of New-York
# we will collect all the points outside the bounding box of newyork city to outlier_locations
outlier_locations = month[((month.dropoff_longitude <= -74.15) | (month.dropoff_latitude <= 40.5774)| \
                   (month.dropoff_longitude >= -73.7004) | (month.dropoff_latitude >= 40.9176))].compute()

# creating a map with the a base location
map_osm = folium.Map(location=[40.734695, -73.990372])

# we will spot only first 100 outliers on the map, plotting all the outliers will take more time
sample_locations = outlier_locations.head(10000)
for i,j in sample_locations.iterrows():
    if int(j['pickup_latitude']) != 0:
        folium.Marker(list((j['dropoff_latitude'],j['dropoff_longitude']))).add_to(map_osm)
map_osm

ValueError: time data "1421349822" doesn't match format "%Y-%m-%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

According to NYC Taxi & Limousine Commision Regulations the maximum allowed trip duration in a 24 hour interval is 12 hours.

In [ ]:
import time
import datetime
import numpy as np
import pandas as pd # Import pandas
import dask.dataframe as dd # Import dask.dataframe

# in our data we have time in the format "YYYY-MM-DD HH:MM:SS" we convert this string to python time format and then into unix time stamp
# Reference Link: https://stackoverflow.com/a/27914405
def convert_to_unix(s):
    # This function is not needed anymore as the data is already in Unix timestamp format
    pass

# we return a data frame which contains the columns
# 1.'passenger_count'
# 2.'trip_distance'
# 3.'pickup_longitude'
# 4.'pickup_latitude'
# 5.'dropoff_longitude'
# 6.'dropoff_latitude'
# 7.'total_amount' : total fair that was paid
# 8.'trip_times' : duration of each trip
# 9.'pickup_times : pickup time converted into unix time
# 10.'Speed' : velocity of each trip
def return_with_trip_times(month):
    # Convert timestamps to datetime objects using Dask, handling both Unix timestamp and string format
    if month['tpep_pickup_datetime'].dtype == 'float64': # Assuming float64 means Unix timestamp
        month['tpep_pickup_datetime_dt'] = dd.to_datetime(month['tpep_pickup_datetime'], unit='s')
        month['tpep_dropoff_datetime_dt'] = dd.to_datetime(month['tpep_dropoff_datetime'], unit='s')
        pickup_times_unix = month['tpep_pickup_datetime'] # Keep original unix timestamp
    else: # Assuming string format 'YYYY-MM-DD HH:MM:SS'
        month['tpep_pickup_datetime_dt'] = dd.to_datetime(month['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
        month['tpep_dropoff_datetime_dt'] = dd.to_datetime(month['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
        # Convert datetime to Unix timestamp for consistency
        pickup_times_unix = month['tpep_pickup_datetime_dt'].astype('int64') // 10**9


    # calculate duration of trips in minutes using Dask
    durations = (month['tpep_dropoff_datetime_dt'] - month['tpep_pickup_datetime_dt']).dt.total_seconds() / 60

    # append durations of trips and speed in miles/hr to a new dataframe
    # Select desired columns first
    new_frame = month[['passenger_count','trip_distance','pickup_longitude','pickup_latitude',
                       'dropoff_longitude','dropoff_latitude','total_amount']]

    # Assign new columns using Dask's assign
    new_frame = new_frame.assign(
        trip_times=durations,
        pickup_times=pickup_times_unix,
        Speed=60*(new_frame['trip_distance']/durations) # Calculate speed using Dask Series
    )

    return new_frame.compute() # Compute the result at the end

# print(frame_with_durations.head())
#  passenger_count      trip_distance   pickup_longitude        pickup_latitude dropoff_longitude       dropoff_latitude        total_amount    trip_times      pickup_times    Speed
#   1                  1.59           -73.993896                40.750111       -73.974785              40.750618               17.05            18.050000      1.421329e+09    5.285319
#   1                   3.30            -74.001648              40.724243       -73.994415              40.759109               17.80           19.833333       1.420902e+09    9.983193
#   1                   1.80            -73.963341              40.802788       -73.951820              40.824413               10.80           10.050000       1.420902e+09    10.746269
#   1                   0.50            -74.009087              40.713818       -74.004326              40.719986               4.80            1.866667        1.420902e+09    16.071429
#   1                   3.00            -73.971176              40.762428       -74.004181              40.742653               16.30           19.316667       1.420902e+09    9.318378

frame_with_durations = return_with_trip_times(month)

In [ ]:
sns.boxplot(y="trip_times", data =frame_with_durations)
plt.show()

In [ ]:
for i in range(90,100):
    var =frame_with_durations["trip_times"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

90 percentile value is 17.866666666666667
91 percentile value is 18.333333333333332
92 percentile value is 18.816666666666666
93 percentile value is 19.366666666666667
94 percentile value is 19.983333333333334
95 percentile value is 20.7
96 percentile value is 21.516666666666666
97 percentile value is 22.533333333333335
98 percentile value is 23.883333333333333
99 percentile value is 26.033333333333335
100 percentile value is  548555.6333333333


In [ ]:
frame_with_durations_modified=frame_with_durations[(frame_with_durations.trip_times>1) & (frame_with_durations.trip_times<720)]

In [ ]:
sns.FacetGrid(frame_with_durations_modified, height=6) \
      .map(sns.kdeplot,"trip_times") \
      .add_legend();
plt.title('KDE Plot of Trip Times (Filtered)')
plt.xlabel('Trip Time (minutes)')
plt.show();

<IPython.core.display.Javascript object>

In [ ]:
import math
frame_with_durations_modified['log_times']=[math.log(i) for i in frame_with_durations_modified['trip_times'].values]

In [ ]:
# check for any outliers in the data after trip duration outliers removed
# box-plot for speeds with outliers
frame_with_durations_modified['Speed'] = 60*(frame_with_durations_modified['trip_distance']/frame_with_durations_modified['trip_times'])
sns.boxplot(y="Speed", data =frame_with_durations_modified)
plt.show()

In [ ]:
#calculating speed values at each percntile 0,10,20,30,40,50,60,70,80,90,100
for i in range(0,100,10):
    var =frame_with_durations_modified["Speed"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))

0 percentile value is 0.0
10 percentile value is 6.239168110918544
20 percentile value is 7.56687898089172
30 percentile value is 8.609865470852018
40 percentile value is 9.56521739130435
50 percentile value is 10.526315789473683
60 percentile value is 11.559633027522937
70 percentile value is 12.772277227722771
80 percentile value is 14.354669464847849
90 percentile value is 16.911627906976744


In [ ]:
#calculating speed values at each percntile 90,91,92,93,94,95,96,97,98,99,100
for i in range(90,100):
    var =frame_with_durations_modified["Speed"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile value is ",var[-1])

90 percentile value is 16.911627906976744
91 percentile value is 17.292134831460675
92 percentile value is 17.711229946524064
93 percentile value is 18.194244604316545
94 percentile value is 18.75
95 percentile value is 19.40869565217391
96 percentile value is 20.212443095599394
97 percentile value is 21.245901639344265
98 percentile value is 22.7027027027027
99 percentile value is 25.11082138200782
100 percentile value is  311.64179104477614


In [ ]:
#calculating speed values at each percntile 99.0,99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100
for i in np.arange(0.0, 1.0, 0.1):
    var =frame_with_durations_modified["Speed"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(99+i,var[int(len(var)*(float(99+i)/100))]))
print("100 percentile value is ",var[-1])

99.0 percentile value is 25.11082138200782
99.1 percentile value is 25.454545454545457
99.2 percentile value is 25.85635359116022
99.3 percentile value is 26.297872340425535
99.4 percentile value is 26.80851063829787
99.5 percentile value is 27.391304347826093
99.6 percentile value is 28.115449915110354
99.7 percentile value is 29.028744326777606
99.8 percentile value is 30.302608695652168
99.9 percentile value is 32.58227848101266
100 percentile value is  311.64179104477614


In [ ]:
frame_with_durations_modified=frame_with_durations_modified[(frame_with_durations_modified.Speed>0) & (frame_with_durations_modified.Speed<45.31)]

In [ ]:
# Calculate mean speed using pandas
mean_speed = frame_with_durations_modified["Speed"].mean()
print(f"Mean speed after filtering: {mean_speed}")

Mean speed after filtering: 11.189153793098281


11.19 mph is the typical speed taxis were moving at during trips included in your filtered dataset.

This seems reasonable for city driving in NYC, considering traffic lights, congestion, and stop-and-go conditions.



In [ ]:
#calculating trip distance values at each percntile 0,10,20,30,40,50,60,70,80,90,100
for i in range(0,100,10):
    var =frame_with_durations_modified["trip_distance"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile value is ",var[-1])

0 percentile value is 0.01
10 percentile value is 0.61
20 percentile value is 0.83
30 percentile value is 1.02
40 percentile value is 1.24
50 percentile value is 1.5
60 percentile value is 1.8
70 percentile value is 2.13
80 percentile value is 2.66
90 percentile value is 3.5
100 percentile value is  6.0


In [ ]:
for i in range(90,100):
    var =frame_with_durations_modified["trip_distance"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile value is ",var[-1])

90 percentile value is 3.5
91 percentile value is 3.6
92 percentile value is 3.74
93 percentile value is 3.9
94 percentile value is 4.04
95 percentile value is 4.2
96 percentile value is 4.4
97 percentile value is 4.67
98 percentile value is 4.93
99 percentile value is 5.3
100 percentile value is  6.0


In [ ]:
#calculating trip distance values at each percntile 99.0,99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100
for i in np.arange(0.0, 1.0, 0.1):
    var =frame_with_durations_modified["trip_distance"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(99+i,var[int(len(var)*(float(99+i)/100))]))
print("100 percentile value is ",var[-1])

99.0 percentile value is 5.3
99.1 percentile value is 5.36
99.2 percentile value is 5.4
99.3 percentile value is 5.46
99.4 percentile value is 5.5
99.5 percentile value is 5.6
99.6 percentile value is 5.64
99.7 percentile value is 5.7
99.8 percentile value is 5.8
99.9 percentile value is 5.9
100 percentile value is  6.0


In [ ]:
frame_with_durations_modified=frame_with_durations[(frame_with_durations.trip_distance>0) & (frame_with_durations.trip_distance<23)]


In [ ]:
#calculating total fare amount values at each percntile 0,10,20,30,40,50,60,70,80,90,100
for i in range(0,100,10):
    var = frame_with_durations_modified["total_amount"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile value is ",var[-1])

0 percentile value is -2.2
10 percentile value is 6.3
20 percentile value is 7.3
30 percentile value is 8.3
40 percentile value is 9.3
50 percentile value is 10.3
60 percentile value is 11.6
70 percentile value is 12.95
80 percentile value is 14.8
90 percentile value is 17.8
100 percentile value is  897.5


In [ ]:
#calculating total fare amount values at each percntile 90,91,92,93,94,95,96,97,98,99,100
for i in range(90,100):
    var = frame_with_durations_modified["total_amount"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile value is ",var[-1])

90 percentile value is 17.8
91 percentile value is 18.3
92 percentile value is 18.8
93 percentile value is 18.96
94 percentile value is 19.55
95 percentile value is 20.0
96 percentile value is 20.75
97 percentile value is 21.35
98 percentile value is 22.4
99 percentile value is 23.75
100 percentile value is  897.5


In [ ]:
#calculating total fare amount values at each percntile 99.0,99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100
for i in np.arange(0.0, 1.0, 0.1):
    var = frame_with_durations_modified["total_amount"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(99+i,var[int(len(var)*(float(99+i)/100))]))
print("100 percentile value is ",var[-1])

In [ ]:
#removing all outliers based on our univariate analysis above
def remove_outliers(new_frame):


    a = new_frame.shape[0]
    print ("Number of pickup records = ",a)
    temp_frame = new_frame[((new_frame.dropoff_longitude >= -74.15) & (new_frame.dropoff_longitude <= -73.7004) &\
                       (new_frame.dropoff_latitude >= 40.5774) & (new_frame.dropoff_latitude <= 40.9176)) & \
                       ((new_frame.pickup_longitude >= -74.15) & (new_frame.pickup_latitude >= 40.5774)& \
                       (new_frame.pickup_longitude <= -73.7004) & (new_frame.pickup_latitude <= 40.9176))]
    b = temp_frame.shape[0]
    print ("Number of outlier coordinates lying outside NY boundaries:",(a-b))


    temp_frame = new_frame[(new_frame.trip_times > 0) & (new_frame.trip_times < 720)]
    c = temp_frame.shape[0]
    print ("Number of outliers from trip times analysis:",(a-c))


    temp_frame = new_frame[(new_frame.trip_distance > 0) & (new_frame.trip_distance < 23)]
    d = temp_frame.shape[0]
    print ("Number of outliers from trip distance analysis:",(a-d))

    temp_frame = new_frame[(new_frame.Speed <= 65) & (new_frame.Speed >= 0)]
    e = temp_frame.shape[0]
    print ("Number of outliers from speed analysis:",(a-e))

    temp_frame = new_frame[(new_frame.total_amount <1000) & (new_frame.total_amount >0)]
    f = temp_frame.shape[0]
    print ("Number of outliers from fare analysis:",(a-f))


    new_frame = new_frame[((new_frame.dropoff_longitude >= -74.15) & (new_frame.dropoff_longitude <= -73.7004) &\
                       (new_frame.dropoff_latitude >= 40.5774) & (new_frame.dropoff_latitude <= 40.9176)) & \
                       ((new_frame.pickup_longitude >= -74.15) & (new_frame.pickup_latitude >= 40.5774)& \
                       (new_frame.pickup_longitude <= -73.7004) & (new_frame.pickup_latitude <= 40.9176))]

    new_frame = new_frame[(new_frame.trip_times > 0) & (new_frame.trip_times < 720)]
    new_frame = new_frame[(new_frame.trip_distance > 0) & (new_frame.trip_distance < 23)]
    new_frame = new_frame[(new_frame.Speed < 45.31) & (new_frame.Speed > 0)]
    new_frame = new_frame[(new_frame.total_amount <1000) & (new_frame.total_amount >0)]

    print ("Total outliers removed",a - new_frame.shape[0])
    print ("---")
    return new_frame

In [ ]:
print ("Removing outliers in the month of Jan-2015")
print ("----")
frame_with_durations_outliers_removed = remove_outliers(frame_with_durations)
print("fraction of data points that remain after removing outliers", float(len(frame_with_durations_outliers_removed))/len(frame_with_durations))

Removing outliers in the month of Jan-2015
----
Number of pickup records =  11183316
Number of outlier coordinates lying outside NY boundaries: 228411
Number of outliers from trip times analysis: 19884
Number of outliers from trip distance analysis: 58175
Number of outliers from speed analysis: 17622
Number of outliers from fare analysis: 1179
Total outliers removed 277114
---
fraction of data points that remain after removing outliers 0.9752207663630358


In [ ]:
#trying different cluster sizes to choose the right K in K-means
coords = frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']].values
neighbours=[]

def find_min_distance(cluster_centers, cluster_len):
    nice_points = 0
    wrong_points = 0
    less2 = []
    more2 = []
    min_dist=1000
    for i in range(0, cluster_len):
        nice_points = 0
        wrong_points = 0
        for j in range(0, cluster_len):
            if j!=i:
                distance = gpxpy.geo.haversine_distance(cluster_centers[i][0], cluster_centers[i][1],cluster_centers[j][0], cluster_centers[j][1])
                min_dist = min(min_dist,distance/(1.60934*1000))
                if (distance/(1.60934*1000)) <= 2:
                    nice_points +=1
                else:
                    wrong_points += 1
        less2.append(nice_points)
        more2.append(wrong_points)
    neighbours.append(less2)
    print ("On choosing a cluster size of ",cluster_len,"\nAvg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2):", np.ceil(sum(less2)/len(less2)), "\nAvg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2):", np.ceil(sum(more2)/len(more2)),"\nMin inter-cluster distance = ",min_dist,"\n---")

def find_clusters(increment):
    kmeans = MiniBatchKMeans(n_clusters=increment, batch_size=10000,random_state=42).fit(coords)
    frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])
    cluster_centers = kmeans.cluster_centers_
    cluster_len = len(cluster_centers)
    return cluster_centers, cluster_len

# we need to choose number of clusters so that, there are more number of cluster regions
#that are close to any cluster center
# and make sure that the minimum inter cluster should not be very less
for increment in range(10, 100, 10):
    cluster_centers, cluster_len = find_clusters(increment)
    find_min_distance(cluster_centers, cluster_len)

On choosing a cluster size of  10 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 3.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 7.0 
Min inter-cluster distance =  0.933537706018889 
---
On choosing a cluster size of  20 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 6.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 14.0 
Min inter-cluster distance =  0.5143043684902525 
---
On choosing a cluster size of  30 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 9.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 21.0 
Min inter-cluster distance =  0.4491487614021052 
---
On choosing a cluster size of  40 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 11.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 29.0 
Min inter-cluster distance 

In [ ]:
# if check for the 50 clusters we can observe that there are two clusters with only 0.3 miles apart from each other
# so we choose 40 clusters for solve the further problem

# Getting 40 clusters using the kmeans
kmeans = MiniBatchKMeans(n_clusters=40, batch_size=10000,random_state=0).fit(coords)
frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])

In [ ]:
# Plotting the cluster centers on OSM
cluster_centers = kmeans.cluster_centers_
cluster_len = len(cluster_centers)
map_osm = folium.Map(location=[40.734695, -73.990372])
for i in range(cluster_len):
    folium.Marker(list((cluster_centers[i][0],cluster_centers[i][1])), popup=(str(cluster_centers[i][0])+str(cluster_centers[i][1]))).add_to(map_osm)
map_osm

In [ ]:
#Visualising the clusters on a map
def plot_clusters(frame):
    city_long_border = (-74.03, -73.75)
    city_lat_border = (40.63, 40.85)
    fig, ax = plt.subplots(ncols=1, nrows=1)
    ax.scatter(frame.pickup_longitude.values[:100000], frame.pickup_latitude.values[:100000], s=10, lw=0,
               c=frame.pickup_cluster.values[:100000], cmap='tab20', alpha=0.2)
    ax.set_xlim(city_long_border)
    ax.set_ylim(city_lat_border)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.show()

plot_clusters(frame_with_durations_outliers_removed)

In [ ]:
print(frame_with_durations_outliers_removed.shape)

(10906202, 11)


In [ ]:
def add_pickup_bins(frame,month,year):
    unix_pickup_times=[i for i in frame['pickup_times'].values]
    unix_times = [[1420070400,1422748800,1425168000,1427846400,1430438400,1433116800],\
                    [1451606400,1454284800,1456790400,1459468800,1462060800,1464739200]]

    start_pickup_unix=unix_times[year-2015][month-1]

    # Rference Link: https://www.timeanddate.com/time/zones/est
    # (int((i-start_pickup_unix)/600)+33) : our unix time is in gmt to we are converting it to est
    tenminutewise_binned_unix_pickup_times=[(int((i-start_pickup_unix)/600)+33) for i in unix_pickup_times]
    frame['pickup_bins'] = np.array(tenminutewise_binned_unix_pickup_times)

    return frame

In [ ]:
# clustering, making pickup bins and grouping by pickup cluster and pickup bins
frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])
jan_2015_frame = add_pickup_bins(frame_with_durations_outliers_removed,1,2015)
jan_2015_groupby = jan_2015_frame[['pickup_cluster','pickup_bins','trip_distance']].groupby(['pickup_cluster','pickup_bins']).count()

In [ ]:
jan_2015_frame.head()


In [ ]:
jan_2015_groupby.head()


In [ ]:
# upto now we cleaned data and prepared data for the month 2015,

# now do the same operations for months Jan, Feb, March of 2016
# 1. get the dataframe which inlcudes only required colums
# 2. adding trip times, speed, unix time stamp of pickup_time
# 4. remove the outliers based on trip_times, speed, trip_duration, total_amount
# 5. add pickup_cluster to each data point
# 6. add pickup_bin (index of 10min intravel to which that trip belongs to)
# 7. group by data, based on 'pickup_cluster' and 'pickuo_bin'

# Data Preparation for the months of Jan,Feb and March 2016
def datapreparation(month,kmeans,month_no,year_no):

    print ("Return with trip times..")

    frame_with_durations = return_with_trip_times(month)

    print ("Remove outliers..")
    frame_with_durations_outliers_removed = remove_outliers(frame_with_durations)

    print ("Estimating clusters..")
    frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])
    #frame_with_durations_outliers_removed_2016['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed_2016[['pickup_latitude', 'pickup_longitude']])

    print ("Final groupbying..")
    final_updated_frame = add_pickup_bins(frame_with_durations_outliers_removed,month_no,year_no)
    final_groupby_frame = final_updated_frame[['pickup_cluster','pickup_bins','trip_distance']].groupby(['pickup_cluster','pickup_bins']).count()

    return final_updated_frame,final_groupby_frame

month_jan_2016 = dd.read_parquet(r'C:\Users\2594j\Downloads\New folder\yellow_tripdata_2016-01.parquet')
month_feb_2016 = dd.read_parquet(r'C:\Users\2594j\Downloads\New folder\yellow_tripdata_2016-02.parquet')
month_mar_2016 = dd.read_parquet(r'C:\Users\2594j\Downloads\New folder\yellow_tripdata_2016-03.parquet')


# Fix: Convert all string columns to plain str/object dtype
for df in [month_jan_2016, month_feb_2016]:
    for col in df.columns:
        if str(df[col].dtype).startswith("string"):
            df[col] = df[col].astype("object")


jan_2016_frame,jan_2016_groupby = datapreparation(month_jan_2016,kmeans,1,2016)
feb_2016_frame,feb_2016_groupby = datapreparation(month_feb_2016,kmeans,2,2016)
mar_2016_frame,mar_2016_groupby = datapreparation(month_mar_2016,kmeans,3,2016)

Return with trip times..
Remove outliers..
Number of pickup records =  10906858
Number of outlier coordinates lying outside NY boundaries: 214677
Number of outliers from trip times analysis: 27190
Number of outliers from trip distance analysis: 79742
Number of outliers from speed analysis: 21047
Number of outliers from fare analysis: 4991
Total outliers removed 297784
---
Estimating clusters..
Final groupbying..
Return with trip times..
Remove outliers..
Number of pickup records =  11382049
Number of outlier coordinates lying outside NY boundaries: 223161
Number of outliers from trip times analysis: 27670
Number of outliers from trip distance analysis: 81902
Number of outliers from speed analysis: 22437
Number of outliers from fare analysis: 5476
Total outliers removed 308177
---
Estimating clusters..
Final groupbying..
Return with trip times..
Remove outliers..
Number of pickup records =  12210952
Number of outlier coordinates lying outside NY boundaries: 232444
Number of outliers fro

In [ ]:
# Gets the unique bins where pickup values are present for each each region

# For each cluster region we will collect all the indices of 10min intervals in which the pickups occurred
# We got an observation that there are some pickup bins that doesn't have any pickups
def return_unq_pickup_bins(frame):
    values = []
    for i in range(0,40):
        new = frame[frame['pickup_cluster'] == i]
        list_unq = list(set(new['pickup_bins']))
        list_unq.sort()
        values.append(list_unq)
    return values

In [ ]:
# for every month we get all indices of 10min intervals in which atleast one pickup occurred

#jan
jan_2015_unique = return_unq_pickup_bins(jan_2015_frame)
jan_2016_unique = return_unq_pickup_bins(jan_2016_frame)

#feb
feb_2016_unique = return_unq_pickup_bins(feb_2016_frame)

#march
mar_2016_unique = return_unq_pickup_bins(mar_2016_frame)



In [ ]:
# For each cluster number of 10-min intervals with 0 pickups
for i in range(40):
    print("for the ",i,"th cluster number of 10min intavels with zero pickups: ",4464 - len(set(jan_2015_unique[i])))
    print('-'*60)

for the  0 th cluster number of 10min intavels with zero pickups:  37
------------------------------------------------------------
for the  1 th cluster number of 10min intavels with zero pickups:  48
------------------------------------------------------------
for the  2 th cluster number of 10min intavels with zero pickups:  38
------------------------------------------------------------
for the  3 th cluster number of 10min intavels with zero pickups:  44
------------------------------------------------------------
for the  4 th cluster number of 10min intavels with zero pickups:  31
------------------------------------------------------------
for the  5 th cluster number of 10min intavels with zero pickups:  557
------------------------------------------------------------
for the  6 th cluster number of 10min intavels with zero pickups:  35
------------------------------------------------------------
for the  7 th cluster number of 10min intavels with zero pickups:  30
------------

In [ ]:
# Fills a value of zero for every bin where no pickup data is present
# the count_values: number pickps that are happened in each region for each 10-min interval
# there wont be any value if there are no picksups.
# values: number of unique bins

# for every 10-min interval(pickup_bin) we will check it is there in our unique bin,
# if it is there we will add the count_values[index] to smoothed data
# if not we add 0 to the smoothed data
# we finally return smoothed data
def fill_missing(count_values,values):
    smoothed_regions=[]
    ind=0
    for r in range(0,40):
        smoothed_bins=[]
        for i in range(4464):
            if i in values[r]:
                smoothed_bins.append(count_values[ind])
                ind+=1
            else:
                smoothed_bins.append(0)
        smoothed_regions.extend(smoothed_bins)
    return smoothed_regions

In [ ]:
# Fills a value of zero for every bin where no pickup data is present
# the count_values: number pickps that has occurred in each region for each 10-min interval
# there wont be any value if there are no picksups.
# values: number of unique bins

# for every 10min intervall(pickup_bin) we will check it is there in our unique bin,
# if it is there we will add the count_values[index] to smoothed data
# if not we add smoothed data (which is calculated based on the methods that are discussed in the above markdown cell)
# we finally return smoothed data
def smoothing(count_values,values):
    smoothed_regions=[] # stores list of final smoothed values of each reigion
    ind=0
    repeat=0
    smoothed_value=0
    for r in range(0,40):
        smoothed_bins=[] #stores the final smoothed values
        repeat=0
        for i in range(4464):
            if repeat!=0: # prevents iteration for a value which is already visited/resolved
                repeat-=1
                continue
            if i in values[r]: #checks if the pickup-bin exists
                smoothed_bins.append(count_values[ind]) # appends the value of the pickup bin if it exists
            else:
                if i!=0:
                    right_hand_limit=0
                    for j in range(i,4464):
                        if  j not in values[r]: #searches for the left-limit or the pickup-bin value which has a pickup value
                            continue
                        else:
                            right_hand_limit=j
                            break
                    if right_hand_limit==0:
                    #Case 1: When we have the last/last few values are found to be missing,hence we have no right-limit here
                        smoothed_value=count_values[ind-1]*1.0/((4463-i)+2)*1.0
                        for j in range(i,4464):
                            smoothed_bins.append(math.ceil(smoothed_value))
                        smoothed_bins[i-1] = math.ceil(smoothed_value)
                        repeat=(4463-i)
                        ind-=1
                    else:
                    #Case 2: When we have the missing values between two known values
                        smoothed_value=(count_values[ind-1]+count_values[ind])*1.0/((right_hand_limit-i)+2)*1.0
                        for j in range(i,right_hand_limit+1):
                            smoothed_bins.append(math.ceil(smoothed_value))
                        smoothed_bins[i-1] = math.ceil(smoothed_value)
                        repeat=(right_hand_limit-i)
                else:
                    #Case 3: When we have the first/first few values are found to be missing,hence we have no left-limit here
                    right_hand_limit=0
                    for j in range(i,4464):
                        if  j not in values[r]:
                            continue
                        else:
                            right_hand_limit=j
                            break
                    smoothed_value=count_values[ind]*1.0/((right_hand_limit-i)+1)*1.0
                    for j in range(i,right_hand_limit+1):
                            smoothed_bins.append(math.ceil(smoothed_value))
                    repeat=(right_hand_limit-i)
            ind+=1
        smoothed_regions.extend(smoothed_bins)
    return smoothed_regions

In [ ]:
#Filling Missing values of Jan-2015 with 0
# here in jan_2015_groupby dataframe the trip_distance represents the number of pickups that are happened
jan_2015_fill = fill_missing(jan_2015_groupby['trip_distance'].values,jan_2015_unique)

#Smoothing Missing values of Jan-2015
jan_2015_smooth = smoothing(jan_2015_groupby['trip_distance'].values,jan_2015_unique)

In [ ]:

# number of 10min indices for jan 2015= 24*31*60/10 = 4464
# number of 10min indices for jan 2016 = 24*31*60/10 = 4464
# number of 10min indices for feb 2016 = 24*29*60/10 = 4176
# number of 10min indices for march 2016 = 24*30*60/10 = 4320
# for each cluster we will have 4464 values, therefore 40*4464 = 178560 (length of the jan_2015_fill)
print("number of 10min intervals among all the clusters ",len(jan_2015_fill))

number of 10min intervals among all the clusters  178560


# Why we choose, these methods and which method is used for which data?

# Let us consider we have data of some month in 2015 jan 1st, 10 _ _ _ 20, i.e there are 10 pickups that occurred in 1st
# 10 10-min interval, 0 pickups in 2nd 10mins interval, 0 pickups in 3rd 10min interval
# and 20 pickups in 4th 10-min interval.
# in fill_missing method we replace these values like 10, 0, 0, 20
# where as in smoothing method we replace these values as 6,6,6,6,6, if we can check the number of pickups
# that are happened in the first 40min are same in both cases, but if you can observe that we looking at the future values
# when you are using smoothing we are looking at the future number of pickups which might cause a data leakage.

# so we use smoothing for jan 2015th data since it acts as our training data
# and we use simple fill_misssing method for 2016th data.

In [ ]:
# Jan-2015 data is smoothed, Jan,Feb & March 2016 data missing values are filled with zero
jan_2015_smooth = smoothing(jan_2015_groupby['trip_distance'].values, jan_2015_unique)
jan_2016_smooth = fill_missing(jan_2016_groupby['trip_distance'].values, jan_2016_unique)
feb_2016_smooth = fill_missing(feb_2016_groupby['trip_distance'].values, feb_2016_unique)
mar_2016_smooth = fill_missing(mar_2016_groupby['trip_distance'].values, mar_2016_unique)

# Making list of all the values of pickup data in every bin for a period of 3 months and storing them region-wise
regions_cum = []

# number of 10min indices for jan 2015= 24*31*60/10 = 4464
# number of 10min indices for jan 2016 = 24*31*60/10 = 4464
# number of 10min indices for feb 2016 = 24*29*60/10 = 4176
# number of 10min indices for march 2016 = 24*31*60/10 = 4464
# regions_cum: it will contain 40 lists, each list will contain 4464 + 4176 + 4464 values which represents the number of pickups
# that occured for three months in 2016 data in each of the 40 clusters/regions of NYC

for i in range(0,40):
    regions_cum.append(jan_2016_smooth[4464*i:4464*(i+1)]+feb_2016_smooth[4176*i:4176*(i+1)]+mar_2016_smooth[4464*i:4464*(i+1)])

# print(len(regions_cum))
# 40
# print(len(regions_cum[0]))
# 13104

In [ ]:
#Preparing the Dataframe only with x(i) values as jan-2015 data and y(i) values as jan-2016
ratios_jan = pd.DataFrame()
ratios_jan['Given']=jan_2015_smooth
ratios_jan['Prediction']=jan_2016_smooth
ratios_jan['Ratios']=ratios_jan['Prediction']*1.0/ratios_jan['Given']*1.0

In [ ]:
# Preparing data to be split into train and test, The below prepares data in cumulative form which will be later split into test and train
# number of 10min indices for jan 2015= 24*31*60/10 = 4464
# number of 10min indices for jan 2016 = 24*31*60/10 = 4464
# number of 10min indices for feb 2016 = 24*29*60/10 = 4176
# number of 10min indices for march 2016 = 24*31*60/10 = 4464
# regions_cum: it will contain 40 lists, each list will contain 4464+4176+4464 values which represents the number of pickups
# that are happened for three months in 2016 data

# print(len(regions_cum))
# 40
# print(len(regions_cum[0]))
# 12960

# we take number of pickups that occurred in last 5 10-min intervals
number_of_time_stamps = 5

# output variable
# it will contain 13099 number of pickups for each cluster
output = []


# tsne_lat will contain 13104-5=13099 times latitude of cluster center for every cluster
# Ex: [[cent_lat 13099times],[cent_lat 13099times], [cent_lat 13099times].... 40 lists]
tsne_lat = []


# tsne_lon will contain 13104-5=13099 times longitude of cluster center for every cluster
# Ex: [[cent_long 13099times],[cent_long 13099times], [cent_long 13099times].... 40 lists]
tsne_lon = []

# we will code each day
# sunday = 0, monday = 1, tue = 2, wed = 3, thur = 4, fri = 5, sat = 6
# for every cluster we will be adding 13099 values, each value represent to which day of the week that pickup bin belongs to
tsne_weekday = []

# its a numpy array, of shape (523960, 5)
# each row corresponds to an entry in out data
# for the first row we will have [f0,f1,f2,f3,f4], where fi = number of pickups occurred in (i+1)th 10-min interval(bin)
# the second row will have [f1,f2,f3,f4,f5]
# the third row will have [f2,f3,f4,f5,f6]
# and so on...
tsne_feature = []


tsne_feature = [0]*number_of_time_stamps

for i in range(0,40):

    tsne_lat.append([kmeans.cluster_centers_[i][0]]*13099)
    tsne_lon.append([kmeans.cluster_centers_[i][1]]*13099)

    # jan 1st 2016 is thursday, so we start our day from 4: "(int(k/144))%7+4"
    # our prediction start from 5th 10min intravel since we need to have number of pickups that are happened in last 5 pickup bins
    tsne_weekday.append([int(((int(k/144))%7+4)%7) for k in range(5,4464+4176+4464)])

    # regions_cum is a list of lists [[x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], .. 40 lsits]
    tsne_feature = np.vstack((tsne_feature, [regions_cum[i][r:r+number_of_time_stamps] for r in range(0,len(regions_cum[i])-number_of_time_stamps)]))

    output.append(regions_cum[i][5:])
tsne_feature = tsne_feature[1:]

In [ ]:
len(tsne_lat[0]) * len(tsne_lat) == tsne_feature.shape[0] == len(tsne_weekday)*len(tsne_weekday[0]) == 40*13099 == len(output)*len(output[0])


True

In [ ]:
# Getting the predictions of exponential moving averages to be used as a feature in cumulative form

# upto now we computed 8 features for every data point that starts from 50th min of the day
# 1. cluster center lattitude
# 2. cluster center longitude
# 3. day of the week
# 4. f_t_1: number of pickups occurred in previous t-1th 10-min interval
# 5. f_t_2: number of pickups occurred in previous t-2th 10-min interval
# 6. f_t_3: number of pickups occurred in previous t-3th 10-min interval
# 7. f_t_4: number of pickups occurred in previous t-4th 10-min interval
# 8. f_t_5: number of pickups occurred in previous t-5th 10-min interval

# from the baseline models we saw the exponential weighted moving average gives us the least error
# we will try to add the same exponential weighted moving average at t as a feature to our data
# exponential weighted moving avarage => p'(t) = alpha*p'(t-1) + (1-alpha)*P(t-1)
alpha=0.3

# it is a temporary array that store exponential weighted moving avarage for each 10-min interval,
# for each cluster it will get reset
# for every cluster it contains 13104 values
predicted_values=[]

# it is similar to tsne_lat
# predict_list is a list of lists [[x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], .. 40 lsits]
predict_list = []

tsne_flat_exp_avg = []

for r in range(0,40):
    for i in range(0,13104):

        if i==0:
            predicted_value= regions_cum[r][0]
            predicted_values.append(0)
            continue

        predicted_values.append(predicted_value)
        predicted_value =int((alpha*predicted_value) + (1-alpha)*(regions_cum[r][i]))

    predict_list.append(predicted_values[5:])
    predicted_values=[]

In [ ]:
# train, test split : 70% 30% split
# Before we start predictions using the tree based regression models we take 3 months of 2016 pickup data
# and split it such that for every region we have 70% data in train and 30% in test,
# ordered date-wise for every region
print("size of train data :", int(13099*0.7))
print("size of test data :", int(13099*0.3))

size of train data : 9169
size of test data : 3929


In [ ]:
# extracting first 9169 timestamp values i.e 70% of 13099 (total timestamps) for our training data
train_features =  [tsne_feature[i*13099:(13099*i+9169)] for i in range(0,40)]

# temp = [0]*(12955 - 9068)
test_features = [tsne_feature[(13099*(i))+9169:13099*(i+1)] for i in range(0,40)]

In [ ]:
print("Number of data clusters",len(train_features), "Number of data points in trian data", len(train_features[0]), "Each data point contains", len(train_features[0][0]),"features")
print("Number of data clusters",len(train_features), "Number of data points in test data", len(test_features[0]), "Each data point contains", len(test_features[0][0]),"features")

Number of data clusters 40 Number of data points in trian data 9169 Each data point contains 5 features
Number of data clusters 40 Number of data points in test data 3930 Each data point contains 5 features


In [ ]:
# extracting first 9169 timestamp values i.e 70% of 13099 (total timestamps) for our training data
tsne_train_flat_lat = [i[:9169] for i in tsne_lat]
tsne_train_flat_lon = [i[:9169] for i in tsne_lon]
tsne_train_flat_weekday = [i[:9169] for i in tsne_weekday]
tsne_train_flat_output = [i[:9169] for i in output]
tsne_train_flat_exp_avg = [i[:9169] for i in predict_list]

In [ ]:
# extracting the rest of the timestamp values i.e 30% of 12956 (total timestamps) for our test data
tsne_test_flat_lat = [i[9169:] for i in tsne_lat]
tsne_test_flat_lon = [i[9169:] for i in tsne_lon]
tsne_test_flat_weekday = [i[9169:] for i in tsne_weekday]
tsne_test_flat_output = [i[9169:] for i in output]
tsne_test_flat_exp_avg = [i[9169:] for i in predict_list]

In [ ]:
# the above contains values in the form of list of lists (i.e. list of values of each region), here we make all of them in one list

train_new_features = []
for i in range(0,40):
    train_new_features.extend(train_features[i])

test_new_features = []

for i in range(0,40):
    test_new_features.extend(test_features[i])

In [ ]:
# converting lists of lists into sinle list i.e flatten
# a  = [[1,2,3,4],[4,6,7,8]]
# print(sum(a,[]))
# [1, 2, 3, 4, 4, 6, 7, 8]

tsne_train_lat = sum(tsne_train_flat_lat, [])
tsne_train_lon = sum(tsne_train_flat_lon, [])
tsne_train_weekday = sum(tsne_train_flat_weekday, [])
tsne_train_output = sum(tsne_train_flat_output, [])
tsne_train_exp_avg = sum(tsne_train_flat_exp_avg,[])

In [ ]:
tsne_test_lat = sum(tsne_test_flat_lat, [])
tsne_test_lon = sum(tsne_test_flat_lon, [])
tsne_test_weekday = sum(tsne_test_flat_weekday, [])
tsne_test_output = sum(tsne_test_flat_output, [])
tsne_test_exp_avg = sum(tsne_test_flat_exp_avg,[])

In [ ]:
# Preparing the data frame for our train data
columns = ['ft_5','ft_4','ft_3','ft_2','ft_1']
df_train = pd.DataFrame(data=train_new_features, columns=columns)
df_train['lat'] = tsne_train_lat
df_train['lon'] = tsne_train_lon
df_train['weekday'] = tsne_train_weekday
df_train['exp_avg'] = tsne_train_exp_avg

print(df_train.shape)

(366760, 9)


In [ ]:
# Preparing the data frame for our test data
df_test = pd.DataFrame(data=test_new_features, columns=columns)
df_test['lat'] = tsne_test_lat
df_test['lon'] = tsne_test_lon
df_test['weekday'] = tsne_test_weekday
df_test['exp_avg'] = tsne_test_exp_avg
print(df_test.shape)

(157200, 9)


In [ ]:
df_test.head()


,ft_5,ft_4,ft_3,ft_2,ft_1,lat,lon,weekday,exp_avg
0,22,14,12,14,15,40.796125,-73.942617,4,14
1,14,12,14,15,10,40.796125,-73.942617,4,11
2,12,14,15,10,26,40.796125,-73.942617,4,21
3,14,15,10,26,19,40.796125,-73.942617,4,19
4,15,10,26,19,17,40.796125,-73.942617,4,17


In [ ]:
# XGBoost Regressor Reference : http://xgboost.readthedocs.io/en/latest/python/python_api.html?#module-xgboost.sklearn
%pip install xgboost
import xgboost as xgb
x_model = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)

x_model.fit(df_train, tsne_train_output)


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\2594j\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
     ---------------------------------------- 0.3/150.0 MB 5.2 MB/s eta 0:00:29
     --------------------------------------- 1.5/150.0 MB 15.4 MB/s eta 0:00:10
      -------------------------------------- 2.8/150.0 MB 19.7 MB/s eta 0:00:08
      -------------------------------------- 3.6/150.0 MB 20.9 MB/s eta 0:00:08
     - ------------------------------------- 4.7/150.0 MB 22.9 MB/s eta 0:00:07
     - ------------------------------------- 6.2/150.0 MB 23.4 MB/s eta 0:00:07
     - ------------------------------------- 7.2/150.0 MB 23.2 MB/s eta 0:00:07
     -- ------------------------------------ 8.2/150.0 MB 22.8 MB/s eta 0:00:07
     -- ------------------------------------ 9.3/150.0 MB 22.0 MB/s eta 0:00:07
     -- ------------------------------------ 9.8/150.0 MB 20.9 MB/s eta 0:00:07
     -- ----------------------------------- 11.1/150.0 MB 22.5 MB/s eta 0:00:07
     --- ---------------------------------- 12.

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [ ]:
#predicting with our trained XG-Boost regressor
# the models x_model is already hyper parameter tuned
# the parameters that we got above are found using grid search

y_pred = x_model.predict(df_test)
xgb_test_predictions = [round(value) for value in y_pred]
y_pred = x_model.predict(df_train)
xgb_train_predictions = [round(value) for value in y_pred]

In [ ]:
train_mape=[]
test_mape=[]
train_mape.append((mean_absolute_error(tsne_train_output, xgb_train_predictions))/(sum(tsne_train_output)/len(tsne_train_output)))
test_mape.append((mean_absolute_error(tsne_test_output, xgb_test_predictions))/(sum(tsne_test_output)/len(tsne_test_output)))
print("Train MAPE: ", train_mape[0])
print("Test MAPE: ", test_mape[0])

Train MAPE:  0.1400024738940679
Test MAPE:  0.13429321789656712
